In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 37.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [3]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
from io import StringIO
import pydotplus

def entropy_calculation(column):
    counts = column.value_counts()
    probs = counts / len(column)
    entropy_val = -sum(probs * np.log2(probs))
    return entropy_val

def average_entropy_calculation(dataset, feature):
    feature_values = dataset[feature].unique()
    avg_entropy = 0

    for val in feature_values:
        subset = dataset[dataset[feature] == val]
        subset_entropy = entropy_calculation(subset['Rank'])
        avg_entropy += (len(subset) / len(dataset)) * subset_entropy

    return avg_entropy

def information_gain_calculation(dataset, feature):
    initial_entropy = entropy_calculation(dataset['Rank'])
    avg_entropy = average_entropy_calculation(dataset, feature)
    info_gain = initial_entropy - avg_entropy
    return info_gain

def decision_tree_accuracy(X_train, X_test, y_train, y_test):
    dt_model = DecisionTreeClassifier(random_state=42)
    dt_model.fit(X_train, y_train)
    predictions = dt_model.predict(X_test)
    accuracy_val = accuracy_score(y_test, predictions)
    return accuracy_val, dt_model

def main():
    dataset = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/dt_data.csv")
    feature = input("Enter the name of a score attribute (Q1 to Q9): ")
    entropy_val = entropy_calculation(dataset['Rank'])
    print(f"Entropy (H): {entropy_val:.3f}")
    avg_entropy = average_entropy_calculation(dataset, feature)
    print(f"Average Entropy (AE) for {feature}: {avg_entropy:.3f}")
    info_gain = information_gain_calculation(dataset, feature)
    print(f"Information Gain (IG) for {feature}: {info_gain:.3f}")
    X = dataset.iloc[:, 2:]
    y = dataset['Rank']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    accuracy_val, dt_model = decision_tree_accuracy(X_train, X_test, y_train, y_test)
    print(f"Decision Tree Accuracy: {accuracy_val:.3f}")
    dot_data = StringIO()
    export_graphviz(dt_model, out_file=dot_data,
                    filled=True, rounded=True,
                    special_characters=True, feature_names=X.columns,
                    class_names=y.unique())
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
    graph.write_png("decision_tree.png")
if __name__ == "__main__":
    main()

Enter the name of a score attribute (Q1 to Q9): Q2
Entropy (H): 0.922
Average Entropy (AE) for Q2: 0.762
Information Gain (IG) for Q2: 0.160
Decision Tree Accuracy: 0.630


In [8]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.impute import SimpleImputer


# Load the dataset
dataset = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/dt_data.csv")

# Split the dataset into features and target
X = dataset.iloc[:, 2:]
y = dataset['Rank']

# Drop rows with missing values
X_clean = X.dropna()
y_clean = y[~X.index.isin(X_clean.index)]

# Perform feature selection and training on the cleaned dataset
selector = SelectKBest(f_classif, k=5)
X_new = selector.fit_transform(X_clean, y_clean)

feature_names = [X.columns[i] for i in selector.get_support(indices=True)]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

# Train the decision tree classifier
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# Evaluate the model using cross-validation
scores = cross_val_score(dt_model, X_new, y, cv=5)
print(f"Cross-Validation Accuracy: {scores.mean():.3f}")

# Evaluate the model on the test set
predictions = dt_model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Test Accuracy: {accuracy:.3f}")

# Plot the decision tree
plt.figure(figsize=(12, 8))
plot_tree(dt_model, feature_names=feature_names, class_names=y.unique(), filled=True)
plt.title("Decision Tree")
plt.savefig("decision_tree.png")

ValueError: Found array with 0 sample(s) (shape=(0, 10)) while a minimum of 1 is required by SelectKBest.